# Detection of human dwellings using segmentation

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import geojson
import time
import sys
import imageio
import pickle
import json
from osgeo import gdal
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tqdm import tqdm
from PIL import Image
from sklearn.decomposition import PCA
from keras.models import Model
from keras.layers import Input, Reshape, Dense, Flatten
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from scipy.spatial.distance import cdist
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from matplotlib.patches import Polygon

2022-09-15 22:41:59.514387: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Add Corners Helper Function

In [2]:
def addCorners(xs,xw,ys,yw,resolution,width,color):
    
    coordinates = np.zeros((5,2))
    coordinates[0,0] = xs
    coordinates[1,0] = xs+width
    coordinates[2,0] = xs+width
    coordinates[3,0] = xs
    coordinates[4,0] = xs
    coordinates[0,1] = ys
    coordinates[1,1] = ys
    coordinates[2,1] = ys+width
    coordinates[3,1] = ys+width
    coordinates[4,1] = ys
    p1 = Polygon(coordinates[:,:2], edgecolor=color,facecolor=color)

    coordinates = np.zeros((5,2))
    coordinates[0,0] = (xw*resolution)+xs
    coordinates[1,0] = (xw*resolution)+xs-width
    coordinates[2,0] = (xw*resolution)+xs-width
    coordinates[3,0] = (xw*resolution)+xs
    coordinates[4,0] = (xw*resolution)+xs
    coordinates[0,1] = ys
    coordinates[1,1] = ys
    coordinates[2,1] = ys+width
    coordinates[3,1] = ys+width
    coordinates[4,1] = ys
    p2 = Polygon(coordinates[:,:2], edgecolor=color,facecolor=color)

    coordinates = np.zeros((5,2))
    coordinates[0,0] = xs
    coordinates[1,0] = xs+width
    coordinates[2,0] = xs+width
    coordinates[3,0] = xs
    coordinates[4,0] = xs
    coordinates[0,1] = (yw*resolution)+ys
    coordinates[1,1] = (yw*resolution)+ys
    coordinates[2,1] = (yw*resolution)+ys-width
    coordinates[3,1] = (yw*resolution)+ys-width
    coordinates[4,1] = (yw*resolution)+ys
    p3 = Polygon(coordinates[:,:2], edgecolor=color,facecolor=color)

    #Corner 4
    coordinates = np.zeros((5,2))
    coordinates[0,0] = (xw*resolution)+xs
    coordinates[1,0] = (xw*resolution)+xs-width
    coordinates[2,0] = (xw*resolution)+xs-width
    coordinates[3,0] = (xw*resolution)+xs
    coordinates[4,0] = (xw*resolution)+xs
    coordinates[0,1] = (yw*resolution)+ys
    coordinates[1,1] = (yw*resolution)+ys
    coordinates[2,1] = (yw*resolution)+ys-width
    coordinates[3,1] = (yw*resolution)+ys-width
    coordinates[4,1] = (yw*resolution)+ys
    p4 = Polygon(coordinates[:,:2], edgecolor=color,facecolor=color)

    #Return
    return p1,p2,p3,p4
    

### Find All Raw Satellite Imagery

In [3]:
testfiles = []
trainfiles = []
basedir = '../../data/spacenet/khartoum/train/raw/RGB-PanSharpen/'
for root, dirs, files in os.walk(basedir, topdown=False):
    for name in files:
        if '.tif' in name:
            trainfiles.append(os.path.join(root, name))
            
basedir = '../../data/spacenet/khartoum/test/raw/RGB-PanSharpen/'
for root, dirs, files in os.walk(basedir, topdown=False):
    for name in files:
        if '.tif' in name:
            testfiles.append(os.path.join(root, name))      

### Loop Through Images to Convert and Create Segmented Masks

In [5]:
resolution = 650
width = 0.000001
count = 0
geobase_dir = '../../data/spacenet/khartoum/train/raw/geojson/buildings/buildings_AOI_5_Khartoum_'
satpng_dir = '../../data/spacenet/khartoum/train/processed/satellite/'
segpng_dir = '../../data/spacenet/khartoum/train/processed/segmented/'
plt.figure(figsize=(12,12))    
fig,ax = plt.subplots()
for i in tqdm(range(len(trainfiles))):
    satfile = trainfiles[i]
    geofile = geobase_dir + satfile.split('_')[-1].replace('.tif','.geojson')
    satpng_file = satpng_dir + str(count).zfill(3) + '.png'
    segpng_file = satpng_file.replace('satellite','segmented')
    satpng_file = satpng_dir + str(count).zfill(3) + '.png'
    segpng_file = satpng_file.replace('satellite','segmented')
    count = count + 2
    raster = gdal.Open(satfile, gdal.GA_ReadOnly)

    band1 = raster.GetRasterBand(1).ReadAsArray()
    band2 = raster.GetRasterBand(2).ReadAsArray()
    band3 = raster.GetRasterBand(3).ReadAsArray()
    im = np.zeros((band1.shape[0],band1.shape[1],3))
    im[:,:,0] = band1
    im[:,:,1] = band2
    im[:,:,2] = band3
    im = im / 2000 * 255
    image = Image.fromarray(im.astype('uint8'))
    image.save(satpng_file)

    geotransform = raster.GetGeoTransform()
    (xs,xw,zs,ys,ze,yw) = geotransform
    xpos = np.linspace(xs,xs+(resolution*xw))
    ypos = np.linspace(ys,ys+(resolution*yw))
    with open(geofile) as f:

        #Add Patches 
        gj = geojson.load(f)
        for item in gj['features']:
            coordinates = item['geometry']['coordinates']
            coordinates = np.squeeze(np.array(coordinates))
            if (len(coordinates.shape) == 2):
                px = coordinates[:,0]
                py = coordinates[:,1]
                p = Polygon(coordinates[:,:2], edgecolor='r',facecolor='r')
                plt.plot(coordinates[:,0],coordinates[:,1],c='r',linewidth=0.1)
                ax.add_patch(p)

    #Add Corners
    p1,p2,p3,p4 = addCorners(xs,xw,ys,yw,resolution,width,'k')
    ax.add_patch(p1)
    ax.add_patch(p2)
    ax.add_patch(p3)
    ax.add_patch(p4)
    plt.box()
    plt.grid('off')
    plt.xticks([])
    plt.yticks([])
    plt.savefig(segpng_file, dpi=300,bbox_inches="tight")    
    plt.clf()